# ENCS5341 - Machine Learning  
## Assignment 1: Data Preprocessing & Exploratory Data Analysis (EDA)

## Environment Setup

In [ ]:
!python -m venv .venv
# and then .venv\Scripts\activate in the terminal

## Install all required packages

In [ ]:
!pip install numpy pandas matplotlib seaborn scikit-learn

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Step 1: Data Loading and Initial Inspection:

### Loading the data

In [ ]:
data = pd.read_csv("../data/Customer_Data.csv")

### Inspecting first few rows

In [ ]:
data.head()

### Checking general information

In [ ]:
data.info()

### Summary statistics

In [ ]:
data.describe()

## Step 2: Handling Missing Data

### Number of null values in each column

In [ ]:
data.isnull().sum()

### Handling missing values for Age

We did not think using the overall mean for the age is ideal, so we decided to fill missing values using the mean for each gender group separately.

In [ ]:
meanAgeFemale = data[data["Gender"] == 1]["Age"].mean()
meanAgeMale = data[data["Gender"] == 0]["Age"].mean()

data.loc[data["Gender"] == 1, "Age"] = data.loc[data["Gender"] == 1, "Age"].fillna(meanAgeFemale)
data.loc[data["Gender"] == 0, "Age"] = data.loc[data["Gender"] == 0, "Age"].fillna(meanAgeMale)

### Handling missing values for SupportCalls
We noticed that the minimum value for SupportCalls is 1, so we assumed that if the value is null it means that no calls were made therefor fill it with zero.

In [ ]:
data["SupportCalls"] = data["SupportCalls"].fillna(0)

### Handling missing values for Income 
Filling the missing income values with the median is not sufficient.

So we grouped customers in bins based on their age range and filled the null income values with the median income of each group

In [ ]:
bins = [10, 20, 30, 40, 50, 60, 70]
labels = ["10-19", "20-29", "30-39", "40-49", "50-59", "60-69"]
data["GroupByAge"] = pd.cut(data["Age"], bins=bins, labels=labels, right=False)

medianPerGroup = data.groupby("GroupByAge")["Income"].median()

for group in medianPerGroup.index:
    data.loc[(data["GroupByAge"] == group) & (data["Income"].isnull()), "Income"] = medianPerGroup[group]

data.drop("GroupByAge", axis=1, inplace=True)

### Handling missing values for Tenure
We filled the missing values based on Churn status and predefined group ratios

In [ ]:
bins = [0, 3, 6, 10]
labels = ['0-2', '3-5', '6-9']

data['TenureGroup'] = pd.cut(data['Tenure'], bins=bins, labels=labels, right=False)
table = pd.crosstab(data['TenureGroup'], data['ChurnStatus'])
print(table)

In [ ]:
table.plot(kind='bar', figsize=(7,5))
plt.title("Churn Status by Tenure Group")
plt.xlabel("Tenure Group (Years)")
plt.ylabel("Number of Customers")
plt.legend(title="Churn Status", labels=["Stayed (0)", "Churned (1)"])
plt.xticks(rotation=0)
plt.show()

When grouping the data by tenure, we notice two things:

        1. All people who churn have been with the company for less than two years
        2. People who don't churn are split among the three groups, we will fill the missing data according to the existing ratio:
            0-2 -> 0.175
            3-5 -> 0.35
            6-9 -> 0.475

In [ ]:
bins = [0, 3, 6, 10]
labels = ['0-2', '3-5', '6-9']

data['TenureGroup'] = pd.cut(data['Tenure'], bins=bins, labels=labels, right=False)
table = pd.crosstab(data['TenureGroup'], data['ChurnStatus'])
data.drop("TenureGroup", axis=1, inplace=True)
table

Using the ratios generated below, we can apply the same ratio to the missing values

In [ ]:
tenureGroupData = pd.cut(data[(data['ChurnStatus'] == 0) & (data['Tenure'].notnull())]['Tenure'], bins=bins, labels=labels, right=False)
ratios = tenureGroupData.value_counts(normalize=True)
ratios

In [ ]:
for i in data.index:
    if pd.isnull(data.loc[i, "Tenure"]):
        if data.loc[i, "ChurnStatus"] == 1:
            data.loc[i, "Tenure"] = np.random.uniform(0, 2)
        else:
            r = np.random.rand()
            if r < 0.175:
                data.loc[i, "Tenure"] = np.random.uniform(0, 2)
            elif r < 0.35:
                data.loc[i, "Tenure"] = np.random.uniform(3, 5)
            else:
                data.loc[i, "Tenure"] = np.random.uniform(6, 9)

### Number of null values in each column after solving missing values

In [ ]:
data.isnull().sum()

## Step 3: Handling Outliers

The following shows the box plot for each numerical feature.

In [ ]:
features = ["Income", "Age", "Tenure", "SupportCalls"]

plt.figure(figsize=(10, 6))

for i, col in enumerate(features, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(y=data[col], color="skyblue")
    plt.title(f"Boxplot of {col}")
    plt.ylabel(col)

plt.tight_layout()
plt.show()

It appears that both Income and SupportCalls contain outlier values. In contrast, Tenure does not seem to have any noticeable outliers. Age also appears generally consistent, although we should further check whether there are any customers belonging to very young age groups.

### Handling outliers in the Age column.


We will determine whether a customer’s age is an outlier by considering both their Age and Tenure.

In [ ]:
outliers = data[data['Age'] - data['Tenure'] <= 15]

print("Count:", len(outliers))

We noticed that there are 199 customers whose Age values were probably entered incorrectly.

To smooth these data, we will generate a random number between 16 and 20 and add it to the Tenure value. The result will be considered as the Age of the customer.

In [ ]:
outliers = (data['Age'] - data['Tenure'] <= 15)

randomNumber = np.random.uniform(16, 20, size=outliers.sum())
data.loc[outliers, 'Age'] = data.loc[outliers, 'Tenure'] + randomNumber

outliersAge = data[data['Age'] - data['Tenure'] <= 15]
print("Count:", len(outliersAge))

The number of Age outliers is now zero. All Age values have been corrected.

### Handling outliers in the SupportCalls column.

The following box plot highlight the presence of outliers in the SupportCalls.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 3))

sns.boxplot(data=data, x="SupportCalls", ax=ax)
ax.set_title("Box Plot of SupportCalls")
plt.show()

It's clear that there are outliers in the SupportCalls. We will detect these rows using the Z-score.

In [ ]:
z_scores = np.abs(stats.zscore(data['SupportCalls'].dropna()))

outliers_mask = z_scores > 3

print("Number of SupportCalls outliers:", outliers_mask.sum())

We will resolve these outliers using binning methods with a bin size of 500, where each value in a bin is replaced by the median of that bin. This smooths the data and reduces the impact of extreme values.

In [ ]:
data_sorted = data.sort_values('SupportCalls').reset_index(drop=True)

bin_size = 500
n = len(data_sorted)
num_bins = int(np.ceil(n / bin_size))

for i in range(num_bins):
    start = i * bin_size
    end = min((i + 1) * bin_size, n)
    median_value = data_sorted.loc[start:end-1, 'SupportCalls'].median()
    data_sorted.loc[start:end-1, 'SupportCalls'] = median_value

data['SupportCalls'] = data_sorted['SupportCalls']

z_scores = np.abs(stats.zscore(data['SupportCalls'].dropna()))

outliers_mask = z_scores > 3

print("Number of SupportCalls outliers after smoothing:", outliers_mask.sum())

SupportCalls values has been smoothed correctly.

### Handling outliers in the Income column.

The following box plot highlight the presence of outliers in the Income.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 3))

sns.boxplot(data=data, x="Income", ax=ax)
ax.set_title("Box Plot of Income")
plt.show()

It's clear that there are outliers in the Income. We will detect these rows using the Z-score.

In [ ]:
z_scores = np.abs(stats.zscore(data['Income'].dropna()))

outliers_mask = z_scores > 3

print("Number of Income outliers:", outliers_mask.sum())

It seems that 50 Income entries were stored in a different unit. To correct this, we will normalize them by dividing each value by 1,000.

In [ ]:
outlier_indices = data['Income'].dropna().index[outliers_mask]

data.loc[outlier_indices, 'Income'] = data.loc[outlier_indices, 'Income'] / 1000

z_scores = np.abs(stats.zscore(data['Income'].dropna()))

outliers_mask = z_scores > 3

print("Number of Income outliers after smoothing:", outliers_mask.sum())

Income values has been smoothed correctly.

After modifying the extreme Income values (by scaling them down), the mean and standard deviation of the column change.

Because Z-scores depend on these statistics, we need to recalculate them to verify whether the adjusted values are now within the normal range or if any points still qualify as outliers. 

So you might have to run the above part twice to clean the Income outliers

The following boxplots confirm that the outliers have been successfully resolved, resulting in more consistent distributions.

In [ ]:
features = ["Income", "Age", "Tenure", "SupportCalls"]

plt.figure(figsize=(10, 6))

for i, col in enumerate(features, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(y=data[col], color="skyblue")
    plt.title(f"Boxplot of {col}")
    plt.ylabel(col)

plt.tight_layout()
plt.show()

## Step 4: Feature Scaling

In feature scaling, we chose Z-score scaling for all of the numerical features we have because:
1. It centers data around mean of 0 and a standard deviation of 1. 
2. Keeps the shape and distribution of each feature.
3. Ensures all of the features are equally important for the model training, even if they have different numerical scales.
4. Our values are not naturally bounded and can be considered continuous.

### Visualize distributions before scaling

In [ ]:
fig, axs = plt.subplots(figsize=(20, 4), ncols=4)
sns.histplot(data=data, x="Age", ax=axs[0], bins=10)
sns.histplot(data=data, x="Income", ax=axs[1], bins=10)
sns.histplot(data=data, x="Tenure", ax=axs[2], bins=10)
sns.histplot(data=data, x="SupportCalls", ax=axs[3], bins=10)

### Apply standardization


In [ ]:
scaler = StandardScaler()
data[['Age', 'Income', 'SupportCalls', 'Tenure']] = scaler.fit_transform(data[['Age', 'Income', 'SupportCalls', 'Tenure']])

### Visualize distributions after scaling

In [ ]:
fig, axs = plt.subplots(figsize=(20, 4), ncols=4)
sns.histplot(data=data, x="Age", ax=axs[0], bins=10)
sns.histplot(data=data, x="Income", ax=axs[1], bins=10)
sns.histplot(data=data, x="Tenure", ax=axs[2], bins=10)
sns.histplot(data=data, x="SupportCalls", ax=axs[3], bins=10)

All numerical features are now standardized with mean 0 and standard deviation 1, ensuring uniform contribution during model training.

## Step 5: Exploratory Data Analysis

### • Univariate Analysis

The following plots are histograms and box plots for all numerical features.

The left column represents the data distribution across different bins.

The right column represents the spread as well as any outliers.

We can make the following remarks based on the plots:

1. For age: the customers are spread uniformly across age distributions with no skew.
2. For income: after removing outliers, the distribution is now more balanced and interpretable, showing the main customer group clearly.
3. For tenure: values are evenly distributed across the full range with no anomalies.
4. For support calls: distribution appears reasonable and concentrated around lower values after removing extreme outliers.

In [ ]:
fig, axs = plt.subplots(figsize=(22, 16), ncols=2, nrows=4)
sns.histplot(data=data, x="Age", ax=axs[0][0], bins=10)
sns.boxplot(data=data, y="Age",ax=axs[0][1])

sns.histplot(data=data, x="Income", ax=axs[1][0], bins=10)
sns.boxplot(data=data, y="Income",ax=axs[1][1])

sns.histplot(data=data, x="Tenure", ax=axs[2][0], bins=10)
sns.boxplot(data=data, y="Tenure",ax=axs[2][1])

sns.histplot(data=data, x="SupportCalls", ax=axs[3][0], bins=10)
sns.boxplot(data=data, y="SupportCalls",ax=axs[3][1])

The following plot shows the distribution of age values separated by gender.

The overlapping bars show how both genders are spread across the age ranges, the colored tips represent which gender occurs more in that age range.

In [ ]:
sns.histplot(data=data, x="Age", hue="Gender", bins=10)

The following bar plots visualize the distribution for all categorical values.

We can make the following remarks based on the plots:

1. For gender: the data is very balanced meaning there is no bias in the distribution.
2. For churn status: the data is not balanced as barely any customers churn compared to the customers who stay.
2. For product type: almost half of the customers go for the premium product rather than the basic. 

In [ ]:
fig, axs = plt.subplots(figsize=(16, 4), ncols=3)
sns.countplot(data=data, x="Gender", ax=axs[0])
sns.countplot(data=data, x="ChurnStatus", ax=axs[1])
sns.countplot(data=data, x="ProductType", ax=axs[2])

### • Bivariate Analysis

The following scatter plot shows the relationship between Age and Income and the points are colored by ChurnStatus.

With the outliers already removed, the relationship between the Income and ChurnStatus becomes clear as no customer with an income higher than 50k has churned across all ages.

In [ ]:
sns.scatterplot(x="Age", y='Income', hue='ChurnStatus', data=data)
plt.title(f"Age vs Income colored by ChurnStatus")

The following plot shows how customers who stayed or churned are distributed across different tenure and income values.

The result comes as no surprise as we already learned the relationship between Tenure and Churn as well as the relationship between Tenure and Income, so this plot is a mix of those.

In [ ]:

sns.scatterplot(x="Tenure", y='Income', hue='ChurnStatus', data=data)
plt.title(f"Tenure vs Income colored by ChurnStatus")

This plot still shows no meaningful pattern between SupportCalls and Income, even after outlier removal, ChurnStatus appears unrelated to these two features.

In [ ]:
sns.scatterplot(x="SupportCalls", y='Income', hue='ChurnStatus', data=data)
plt.title(f"SupportCalls vs Income colored by ChurnStatus")

The following box plots show the relationship between numerical features and ChurnStatus.

The plots shows that customers who churn have lower tenure and lower income.

In [ ]:
fig, axs = plt.subplots(figsize=(24, 4), ncols=4)
sns.boxplot(x="ChurnStatus", y="Age", data=data, ax=axs[0])
sns.boxplot(x="ChurnStatus", y="Income", data=data, ax=axs[1])
sns.boxplot(x="ChurnStatus", y="Tenure", data=data, ax=axs[2])
sns.boxplot(x="ChurnStatus", y="SupportCalls", data=data, ax=axs[3])

The following count plots show how Gender and ProductType relate to the ChurnStatus.

Since the churn ratio appears similar for customers who churned or who leaved in both categories, we can conclude that Gender and ProductType play no part in predicting churn status.

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4), ncols=2)
sns.countplot(x='Gender', hue='ChurnStatus', data=data, ax=axs[0])
sns.countplot(x='ProductType', hue='ChurnStatus', data=data, ax=axs[1])

### • Correlation Analysis

The following plot contains all of the plots for all of the features against each other. It helps us get a single massive picture for all of the features at once. 

In [ ]:
sns.pairplot(data[['Age','Income','Tenure','SupportCalls','ChurnStatus']], hue='ChurnStatus')

We can conclude from the following heatmap that the correlation between Tenure and the target is the strongest and we can split the data to two categorizes:

1. Customers who have been with the company longer tend to stay.
2. New customers are possible to churn.

Other features do not affect the target feature as their correlations are close to 0, showing that they have weak affect on churn in this data.

In [ ]:
sns.heatmap(data[['Tenure', 'Age', 'Income', 'SupportCalls', 'ChurnStatus']].corr(), annot=True)
plt.title('Correlation Heatmap')

## Step 6: Data Visualization

### Age vs Income vs ChurnStatus

In [ ]:
sns.scatterplot(x="Age", y='Income', hue='ChurnStatus', data=data)
plt.title(f"Age vs Income colored by ChurnStatus")

This plot represents how income is distributed for the different ages of the customers and which of them have churned or stayed.

### Tenure vs ChurnStatus

In [ ]:
sns.boxplot(x='ChurnStatus', y='Tenure', data=data)
plt.title(f"Tenure vs ChurnStatus boxplot")

The plot shows how customers with higher income tend to stay, while low income customers are more likely to churn.

### ProductType vs ChurnStatus

In [ ]:
sns.countplot(x='ProductType', hue='ChurnStatus', data=data)
plt.title("ProductType vs ChurnStatus")

This plot compares churn status across the two product types and shows that the product type has no influence on whether a customer churns or stays.

### Gender vs ChurnStatus

In [ ]:
sns.countplot(hue='Gender',x='ChurnStatus', data=data)
plt.title("Gender vs ChurnStatus")

The plot shows the relationship between a customer's gender and whether they churn or not. It's visible that there is no relationship between the two.  

## Step 7:  Ending Remarks

- The dataset was successfully cleaned and all missing values were handled.
- Outliers were checked for each feature and were detected in *Income*, *Age*, and *SupportCalls*.
- **Tenure** has the strongest correlation and affect on whether a customers churns or stays. 
    - Customers who stay for the first two years, do not churn.
- **Income** also has an influence on the likelihood of churning.
    - Customers who make more than 50k do not churn.
- **Gender**, **ProductType** and **Age** have little effect on churn.
- These visualizations, statistics and insights will help us build a good predictive model in the future.